# Домашнее задание к лекции «Основы веб-скрапинга и работы с API»

## Преподаватель: Олег Булыгин
## Студент: Сотский Григорий

### Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import time
from datetime import datetime, timedelta

In [2]:
KEYWORDS = ['python', 'парсинг']

In [3]:
now = datetime.now()
today = '{:%d-%m-%Y}'.format(now, now.day)
yesterday = '{:%d-%m-%Y}'.format(now, now.day - 1)

In [4]:
req = requests.get('https://habr.com/ru/')
soup = bs(req.text, 'html.parser')

In [5]:
posts = soup.find_all('li', class_='content-list__item_post')

In [6]:
habr = pd.DataFrame() 
for post in posts:
    post_attrs = post.attrs
    try:
        post_id = post_attrs['id']
        post_id = str(post_id.split('_')[-1])
    except:
        continue
    link = 'https://habr.com/ru/post/' + post_id
    titles = post.find_all('a', class_='post__title_link')
    for title in titles:
        title = title.text.lower()
    date_post = post.find('span', class_='post__time')
    for date in date_post:
        if 'сегодня' in date:
            date = today
        elif 'вчера' in date:
            date = yesterday
    preview_texts = post.find_all('div', class_='post__text')
    for preview_text in preview_texts:
        preview_text = preview_text.text.lower()
        for desired in KEYWORDS:
            if desired in preview_text:
                req2 = requests.get(link)
                soup2 = bs(req2.text, 'html.parser')
                full_post = soup2.find_all('div', class_='post__text')
                for full_text in full_post:
#                     full_text = full_text.text.lower()
                    row = {'date': date, 'title': title, 'link': link, 'text': full_text}
                    habr = pd.concat([habr, pd.DataFrame([row])])

In [7]:
habr

,date,title,link,text
0,16-07-2020,надоел javascript — используй браузерный python,https://habr.com/ru/post/511146,[[Мой опыт разработки игры «Змейка» на Brython...
0,16-07-2020,как в компании развивался python. доклад яндекса,https://habr.com/ru/post/509352,[13 лет назад начался эксперимент по использов...


### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods

In [8]:
EMAIL = ['gregory@ya.ru', 'gregory@yandex.ru']

In [15]:
for email in EMAIL:
    post_req = requests.post('https://digibody.avast.com/v1/web/leaks', json = {'email' : email})
    avast = pd.DataFrame(post_req.json()['value'])
avast

,leak_id,username,domain,passwords,leak_info,marked_resolved,leak_date,passwords_count
0,037a277e-d484-4c34-8473-904024e489e1,gregory@yandex.ru,,"[{'password': None, 'encrypted': False}]","{'id': '037a277e-d484-4c34-8473-904024e489e1',...",False,1513900800000,1
1,8908dde7-dc5b-44a7-8720-d6090c933497,gregory@yandex.ru,,"[{'password': None, 'encrypted': False}]","{'id': '8908dde7-dc5b-44a7-8720-d6090c933497',...",False,1535068800000,1
2,cf7afbbb-7695-4f97-b755-3d63df87828f,gregory@yandex.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'cf7afbbb-7695-4f97-b755-3d63df87828f',...",False,1549411200000,1
3,c0338e81-fce6-450c-ab32-559867d243c2,gregory@yandex.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'c0338e81-fce6-450c-ab32-559867d243c2',...",False,1507507200000,1
4,71db3f29-006d-4158-a3f0-8102652fbf57,gregory@yandex.ru,,"[{'password': None, 'encrypted': False}]","{'id': '71db3f29-006d-4158-a3f0-8102652fbf57',...",False,1549411200000,1
5,83571809-585b-4fff-a72f-cdf0bc345916,gregory@yandex.ru,,"[{'password': None, 'encrypted': False}]","{'id': '83571809-585b-4fff-a72f-cdf0bc345916',...",False,1549411200000,1
6,1631f5e3-9b31-4a78-be91-111a01a87b56,grigiory,,"[{'password': None, 'encrypted': False}]","{'id': '1631f5e3-9b31-4a78-be91-111a01a87b56',...",False,1576195200000,1
7,70160173-5bba-4336-8b4f-5b0eb6959857,gregory@yandex.ru,stratfor.com,"[{'password': None, 'encrypted': True}]","{'id': '70160173-5bba-4336-8b4f-5b0eb6959857',...",False,1477180800000,1
8,948e8397-ad5b-4a56-99bb-b8aa4833bfe1,gregory@yandex.ru,myheritage.com,"[{'password': None, 'encrypted': True}]","{'id': '948e8397-ad5b-4a56-99bb-b8aa4833bfe1',...",False,1509753600000,1
9,b15edc13-85b2-4c53-a54a-fee2d6b861ea,gregory@yandex.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'b15edc13-85b2-4c53-a54a-fee2d6b861ea',...",False,1548720000000,1


In [16]:
avast['leak_date'] = pd.to_datetime(avast['leak_date'], unit='ms')

In [17]:
avast.rename(columns={'leak_id': 'id'}, inplace=True)
avast.rename(columns={'username': 'E-mail'}, inplace=True)
avast.rename(columns={'leak_date': 'date'}, inplace=True)

In [18]:
avast = avast.loc[:, ['id', 'E-mail', 'date']]
avast

,id,E-mail,date
0,037a277e-d484-4c34-8473-904024e489e1,gregory@yandex.ru,2017-12-22
1,8908dde7-dc5b-44a7-8720-d6090c933497,gregory@yandex.ru,2018-08-24
2,cf7afbbb-7695-4f97-b755-3d63df87828f,gregory@yandex.ru,2019-02-06
3,c0338e81-fce6-450c-ab32-559867d243c2,gregory@yandex.ru,2017-10-09
4,71db3f29-006d-4158-a3f0-8102652fbf57,gregory@yandex.ru,2019-02-06
5,83571809-585b-4fff-a72f-cdf0bc345916,gregory@yandex.ru,2019-02-06
6,1631f5e3-9b31-4a78-be91-111a01a87b56,grigiory,2019-12-13
7,70160173-5bba-4336-8b4f-5b0eb6959857,gregory@yandex.ru,2016-10-23
8,948e8397-ad5b-4a56-99bb-b8aa4833bfe1,gregory@yandex.ru,2017-11-04
9,b15edc13-85b2-4c53-a54a-fee2d6b861ea,gregory@yandex.ru,2019-01-29


In [19]:
value = post_req.json()['value']
value

[{'leak_id': '037a277e-d484-4c34-8473-904024e489e1',
  'username': 'gregory@yandex.ru',
  'domain': '',
  'passwords': [{'password': None, 'encrypted': False}],
  'leak_info': {'id': '037a277e-d484-4c34-8473-904024e489e1',
   'date': 1513900800000,
   'title': 'Combolist of 1.4 Billion Credentials',
   'description': 'The proliferation of stolen or leaked databases has given rise to credential stuffing, a fairly simple technique in which criminals load lists of stolen credentials, called combo lists, into automated brute-forcing tools to test credentials en masse. These tools test stolen passwords against thousands of targeted websites and applications until there is a match.\n\nThis particular combo list was likely compiled over time from a variety of public and private breaches. It contains approximately 1.4 billion email and password records. Criminals are actively leveraging this list, along with credential stuffing tools, to gain unauthorized access to targeted websites. This list

In [20]:
avast_ = pd.DataFrame() 
for items in value:
    for item in items.values():
        avast_ = pd.concat([avast_, pd.DataFrame([item])])

In [21]:
avast_

,0,date,description,domains,id,media_references,number_of_entries,picture_url,service_name,source_references,title
0,037a277e-d484-4c34-8473-904024e489e1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,gregory@yandex.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"{'password': None, 'encrypted': False}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,1.513901e+12,The proliferation of stolen or leaked database...,[],037a277e-d484-4c34-8473-904024e489e1,[],1.335492e+09,None,None,[],Combolist of 1.4 Billion Credentials
...,...,...,...,...,...,...,...,...,...,...,...
0,"{'password': None, 'encrypted': False}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,1.477958e+12,"In 2011, Russian instant messaging service pro...",[qip.ru],b06f585f-ccc3-48f0-8bf8-c0ebddfe400b,[https://uk.news.yahoo.com/2016-mega-breaches-...,2.927101e+07,None,None,[],Qip
0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1477958400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
id_ = avast_.id.dropna().reset_index()
title = avast_.title.dropna().reset_index()
description = avast_.description.dropna().reset_index()
media_references = avast_.media_references.dropna().reset_index()
id_ = id_.loc[:, ['id']]
title = title.loc[:, ['title']]
description = description.loc[:, ['description']]
media_references = media_references.loc[:, ['media_references']]
avast__ = pd.concat([id_, title], axis=1)
avast__ = pd.concat([avast__, description], axis=1)
avast__ = pd.concat([avast__, media_references], axis=1)
avast = avast.merge(avast__, how='left', on='id')
avast

,id,E-mail,date,title,description,media_references
0,037a277e-d484-4c34-8473-904024e489e1,gregory@yandex.ru,2017-12-22,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...,[]
1,8908dde7-dc5b-44a7-8720-d6090c933497,gregory@yandex.ru,2018-08-24,Jadid List from Pemiblanc.com,This combolist was compiled from a variety of ...,[]
2,cf7afbbb-7695-4f97-b755-3d63df87828f,gregory@yandex.ru,2019-02-06,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix...",[]
3,c0338e81-fce6-450c-ab32-559867d243c2,gregory@yandex.ru,2017-10-09,Exploit.in combolist,The proliferation of stolen or leaked database...,[]
4,71db3f29-006d-4158-a3f0-8102652fbf57,gregory@yandex.ru,2019-02-06,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix...",[]
5,83571809-585b-4fff-a72f-cdf0bc345916,gregory@yandex.ru,2019-02-06,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix...",[]
6,1631f5e3-9b31-4a78-be91-111a01a87b56,grigiory,2019-12-13,Sensitive Source,This source has been marked as sensitive due t...,[]
7,70160173-5bba-4336-8b4f-5b0eb6959857,gregory@yandex.ru,2016-10-23,Stratfor,"In December 2011, a group of hackers called An...",[http://www.nytimes.com/2011/12/26/technology/...
8,948e8397-ad5b-4a56-99bb-b8aa4833bfe1,gregory@yandex.ru,2017-11-04,MyHeritage,"In October 2017, a customer database belonging...",[]
9,b15edc13-85b2-4c53-a54a-fee2d6b861ea,gregory@yandex.ru,2019-01-29,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix...",[]


### Задание 3 
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods

GROUP = 'netology’
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [23]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/newsfeed.get?'
TOKEN = 'del'
VERSION = '5.120'
FILTERS = 'post'
SOURCE_ID = '-30159897'
SLEEP = 0.33

In [24]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'filters': FILTERS,
    'source_ids': SOURCE_ID,
    'count': 50
}

In [25]:
res = requests.get(NEWSFEED_REQUEST, params)
res

<Response [200]>

In [26]:
res.json()['response']['items']

[{'can_doubt_category': False,
  'can_set_category': False,
  'type': 'post',
  'source_id': -30159897,
  'date': 1594823220,
  'post_type': 'post',
  'text': 'Как стать программистом? Разберёмся на бесплатном курсе-путеводителе для всех, кто хочет попробовать себя в разработке, но не знает, с чего начать.\n\nЧто в программе:\n🔸 Направления разработки: выясним, как проходят рабочие будни QA, frontend-ов, DevOps-ов, desktop и mobile-разработчиков.\n🔸 Языки программирования: после занятия сможете отстоять в споре свой любимый. А ещё попробуете писать код и поймёте, какой язык программирования учить первым.\n🔸 Как устроиться на первую работу: стажировки, вакансии, грейды, фриланс и коварные вопросы на собеседованиях.\n\nСтартуем 27 июля → http://netolo.gy/fFI',
  'marked_as_ads': 0,
  'attachments': [{'type': 'link',
    'link': {'url': 'http://netolo.gy/fFI',
     'title': '«Как стать программистом» – бесплатный онлайн интенсив',
     'caption': 'netology.ru',
     'description': 'Беспла

In [27]:
vk = pd.DataFrame(res.json()['response']['items'])
vk.head(1)

,can_doubt_category,can_set_category,type,source_id,date,post_type,text,marked_as_ads,attachments,post_source,comments,likes,reposts,views,is_favorite,post_id
0,False,False,post,-30159897,1594823220,post,Как стать программистом? Разберёмся на бесплат...,0,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 5, 'user_likes': 0, 'can_like': 1, '...","{'count': 1, 'user_reposted': 0}",{'count': 1605},False,35959


In [28]:
vk['date'] = pd.to_datetime(vk['date'], unit='s')
vk = vk.loc[:, ['date', 'text']]
vk

,date,text
0,2020-07-15 14:27:00,Как стать программистом? Разберёмся на бесплат...
1,2020-07-15 07:27:00,Пишите свои варианты ответов в комментарии. А ...
2,2020-07-14 15:17:00,Отказываться бывает сложно — мы боимся выгляде...
3,2020-07-14 07:47:00,"Что делать, если работа не приносит ни радости..."
4,2020-07-13 07:50:00,🔥 Запустили бесплатный курс по основам работы ...
5,2020-07-12 16:42:00,"Подборка душевных фильмов, которые помогают не..."
6,2020-07-12 08:13:23,"Тополиный пух, жара, июль и снова бесплатные к..."
7,2020-07-11 20:47:24,"Начинать новую жизнь принято с понедельника, н..."
8,2020-07-09 16:42:00,"Можно отлично разбираться в теме, написать отл..."
9,2020-07-09 11:31:15,"Иногда сидишь перед пустым документом, а в гол..."
